# Approximation via Shortest Paths 

Uses the shortest path distances between nodes. 
Very fast but inaccurate model for spreading on networks. 

We compare these approximations against simulation data from 1000 runs.

## Setup and Imports

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import altair as alt
import networkx as nx

from origins_of_the_fittest.network_construction.load_example_networks import (
    load_network_barabasi_albert_Meq4N,
)

from origins_of_the_fittest.approximation.distance_shortest_path import (
    get_sp_distance_unweighted,
)

from origins_of_the_fittest.approximation.distance_message_passing import (
    si_dynamic_message_passing_unweighted,
)

from origins_of_the_fittest.approximation.tau_h_integrals import (
    compute_H_integral_from_ts_normalized,
    compute_tau_integral_from_ts_normalized,
)

from origins_of_the_fittest.approximation.pi_computation import H2pi

from origins_of_the_fittest.utility.plot_altair import altair_helvetica_theme

In [ ]:
# Configure Altair theme
# switch between "light" and "dark" for better visibility in light/dark mode
altair_helvetica_theme("light")

## Load Data and Network

We load:
- Simulation data from 1000 runs with mutation rate μ = 10⁻³
- Barabási-Albert network with N = 1024 nodes and M = 4N edges

In [3]:
# Load network data
A_ba1024 = load_network_barabasi_albert_Meq4N(1024)


# Load simulation data
file_path = "df_simulation_tau_pi_ba1024mu1e-3.parquet"
df_adaptive_ba1024 = (
    pd.read_parquet(file_path).sort_values("origin").reset_index(drop=True)
)

df_adaptive_ba1024.head()

,origin,tau_simulation,tau_simulation_inv,pi_simulation
0,0,8.724844,0.114615,0.004312
1,1,11.109534,0.090013,0.002620
2,2,10.512971,0.095121,0.003024
3,3,10.298010,0.097106,0.003115
4,4,11.983190,0.083450,0.002387


In [4]:
alt.Chart(df_adaptive_ba1024).mark_point().encode(
    x=alt.X("tau_simulation_inv", scale=alt.Scale(type="linear", zero=False)),
    y=alt.Y("pi_simulation", scale=alt.Scale(type="linear")),
)

alt.Chart(...)

Shortest Path Approximation

The shortest path approximation assumes that spread follows the shortest paths on the network. For each origin node, we:
1. Compute shortest path distances to all other nodes
2. Use these distances as arrival times (scaled by s_0)
3. Compute the H integral
4. Compute the steady state to get the probability distribution

In [5]:
# Compute shortest path distances
G_ba1024 = nx.from_pandas_adjacency(A_ba1024)
df_distance_ba1024 = get_sp_distance_unweighted(G_ba1024)

# parameters
mu = 1e-3
s_eff = 0.2

In [6]:
# Compute H matrix using shortest path approximation
N = A_ba1024.shape[0]
H_matrix_sp = np.zeros((N, N))

print("Computing H matrix for all origin nodes using shortest path approximation...")
for j in range(N):
    # Get shortest path distances from node j to all destinations i
    distances = df_distance_ba1024.loc[:, j].to_numpy()

    # Compute H integral: H[:, j] represents destinations given origin j
    H_matrix_sp[:, j] = compute_H_integral_from_ts_normalized(
        distances * 8,  # factor 8 for weighted a = 1/mean(degree)
        destinations=df_distance_ba1024.index.to_numpy(),
        mu=mu,
        s_eff=s_eff,
    )

df_H_sp = pd.DataFrame(
    H_matrix_sp,
    index=pd.Index(range(N), name="i"),  # i = destination
    columns=pd.Index(range(N), name="j"),  # j = origin
)


print("Done!")

Computing H matrix for all origin nodes using shortest path approximation...
Done!


In [7]:
# Convert H matrix to adaptive mutation distribution
df_pi_sp = H2pi(df_H_sp).to_frame(name="pi_sp")

## Shortest Path Approximation vs. Simulation Results

In [8]:
alt.Chart(df_adaptive_ba1024.join(df_pi_sp)).mark_point().encode(
    x=alt.X("pi_sp").title("π (Approximation, shortest path)"),
    y=alt.Y("pi_simulation").title("π (Simulation)"),
).properties(
    title=alt.TitleParams(
        "Shortest Path Approximation π vs Simulation π",
        subtitle=[
            "The shortest path approximation of π is highly correlated to π",
            "but overestimates heterogeneity. ",
        ],
        subtitleColor="#666",
    )
)

alt.Chart(...)